In [1]:
# Imports:
import sys
import warnings
from os import getcwd

import datetime as dt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path

from skyfield.api import EarthSatellite, load
from sgp4.api import Satrec, WGS72
from sgp4.model import wgs72, wgs84
from sgp4.conveniences import jday_datetime, UTC, sat_epoch_datetime, dump_satrec
from sgp4 import exporter

# Load the GMAT Python API
sys.path.append('C:/Work/Programme/GMAT_R2022a/GMAT/api/')
from load_gmat import *

# Load helper functions
#from src.functions import *

In [2]:
# %pip install astropy --upgrade --user

In [3]:
# load ephemeris/gps data
filename = "SP_87"
sat_name = "tubin_2bin"
gps_data = pd.read_csv(getcwd() + "/ephemeris_gps/" + filename + ".csv",sep=";")

In [4]:
# process ephemeris/gps data
gps_data["Position (TOD) Y [km]"].iloc[0]
gps_data_headers = list(gps_data)
print(gps_data_headers)

# only use gps_data where navigation status = 2 
gps_data = gps_data[gps_data["GPS navigation status"] != 0]
gps_data = gps_data[gps_data["GPS navigation status"] != 255]

# convert to datetime 
gps_data["Timestamp [UTC]"] = pd.to_datetime(gps_data["Timestamp [UTC]"])
print(gps_data["Timestamp [UTC]"])

['Timestamp [UTC]', 'Position (TOD) X [km]', 'Position (TOD) Y [km]', 'Position (TOD) Z [km]', 'Velocity (TOD) X [m/s]', 'Velocity (TOD) Y [m/s]', 'Velocity (TOD) Z [m/s]', 'Navigation solution epoch UTC', 'GPS navigation status', 'Number of tracked satellites', 'Position dilution of precision', 'Position (interpolated) (TOD) X [km]', 'Position (interpolated) (TOD) Y [km]', 'Position (interpolated) (TOD) Z [km]', 'Velocity (interpolated) (TOD) X [m/s]', 'Velocity (interpolated) (TOD) Y [m/s]', 'Velocity (interpolated) (TOD) Z [m/s]']
3   2024-05-15 15:05:47.000
4   2024-05-15 15:06:16.800
5   2024-05-15 15:06:46.800
6   2024-05-15 15:07:16.800
7   2024-05-15 15:07:46.800
8   2024-05-15 15:08:16.800
Name: Timestamp [UTC], dtype: datetime64[ns]


In [5]:
#from astropy.coordinates import TEME # True Equator Mean Equinox
#from astropy.coordinates import TETE # True of date / True Equator True Equinox
# https://docs.astropy.org/en/stable/coordinates/index.html#module-astropy.coordinates.builtin_frames
# conversion from TETE (TOD) to TEME

import astropy

#from astropy.coordinates import SkyCoord
#from astropy.table import QTable

In [ ]:
pos = SkyCoord(gps_data["Position (TOD) X [km]"].iloc[:],gps_data["Position (TOD) Y [km]"].iloc[:],gps_data["Position (TOD) Z [km]"].iloc[:], representation_type = 'cartesian', frame = 'tete', unit = 'km')
pos.transform_to('teme')
pos = QTable([pos],names = ['skycoord']) 
pos.write('skycoord_pos.ecsv',overwrite = True)
pos = pd.read_csv('skycoord_pos.ecsv', comment='#')
#pos.rename(columns={"skycoord.x" : "pos_x", "skycoord.y" : "pos_y","skycoord.z" : "pos_z"})

vel = SkyCoord(gps_data["Velocity (TOD) X [m/s]"].iloc[:],gps_data["Velocity (TOD) Y [m/s]"].iloc[:],gps_data["Velocity (TOD) Z [m/s]"].iloc[:], representation_type = 'cartesian', frame = 'tete', unit = 'm/s')
vel.transform_to('teme')
vel = QTable([vel],names = ['skycoord']) 
vel.write('skycoord_vel.ecsv',overwrite = True)
vel = pd.read_csv('skycoord_vel.ecsv', comment='#')
#vel.rename(columns={"skycoord.x" : "vel_x", "skycoord.y" : "vel_y","skycoord.z" : "vel_z"})

In [ ]:
# fit ephemeris/gps to data

# reference frames: https://ai-solutions.com/_help_Files/orbit_reference_frames.htm
# Astropy convert transformations: https://docs.astropy.org/en/stable/coordinates/transforming.html
# TEME reference frame in astropy: https://docs.astropy.org/en/stable/api/astropy.coordinates.TEME.html
# needs to be rotated by the angle between mean vernal equinox at the current epoch and true vernal equinox at the current epoch

# need to convert from TOD to TEME (Equinox)
# https://beyond.readthedocs.io/en/latest/api/frames.html

t = gps_data["Timestamp [UTC]"]
ephemeris = [((row["Position (TOD) X [km]"], row["Position (TOD) Y [km]"], row["Position (TOD) Z [km]"]), (row["Velocity (TOD) X [m/s]"], row["Velocity (TOD) Y [m/s]"], row["Velocity (TOD) Z [m/s]"])) for idx, row in gps_data.iterrows()]

ephemeris_teme = ITRF2TEME(t, ephemeris)
ephemeris = ephemeris_teme

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    t = np.array([_t.to_pydatetime() for _t in t])

# Run the fitter
last_obs = 4320
obs_stride = 1
epoch_obs = 0
lamda = 1e-3 * 0 + 1 # Interesting.The smaller number works, but diverges. This is better
rms_epsilon = 0.002
iterations, solve_sat, elements_coe, sigma, sigmas, dxs, bs, lamdas, b_epoch, b_new_epoch, b, P, A = \
test_tle_fit_normalized_equinoctial(t, ephemeris, central_diff=True, last_obs=last_obs, obs_stride=obs_stride, epoch_obs=epoch_obs, lamda=lamda, rms_epsilon=rms_epsilon, debug=False)

# Optionally thin the observations
tt = t[::obs_stride]
tephemeris = ephemeris[::obs_stride]

if last_obs:
    tt = tt[:last_obs]
    tephemeris = tephemeris[:last_obs]


tle = '\n'.join(exporter.export_tle(solve_sat.model))
print(tle)


/home/student/Desktop/space_ops/src/functions.py:360: RuntimeWarning: invalid value encountered in sqrt
  n = np.sqrt(wgs72.mu / a**3) * 60  # radians / min
/home/student/Desktop/space_ops/src/functions.py:365: RuntimeWarning: invalid value encountered in sqrt
  period = 2 * np.pi * np.sqrt(a**3 / wgs72.mu) / 60  # minutes
/home/student/Desktop/space_ops/src/functions.py:387: RuntimeWarning: invalid value encountered in sqrt
  variances[3:] /= np.sqrt(wgs72.mu / orig_elements[0])
/home/student/Desktop/space_ops/src/functions.py:392: RuntimeWarning: invalid value encountered in sqrt
  b_scale[3:] /= np.sqrt(wgs72.mu / orig_elements[0])
/home/student/Desktop/space_ops/src/functions.py:35: RuntimeWarning: invalid value encountered in sqrt
  n = np.sqrt(wgs72.mu / a**3)
/home/student/Desktop/space_ops/src/functions.py:479: RuntimeWarning: invalid value encountered in sqrt
  A[3:6, idx] = res[1] / np.sqrt(wgs72.mu / orig_elements[0])


LinAlgError: Eigenvalues did not converge